# CS155 Miniproject 1

zchen@caltech.edu

Sentiment Analysis via Neural Networks

In [64]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

import sklearn
import sklearn.model_selection

import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization, LeakyReLU

## Loading Dataset and preprocessing

In [141]:
# Load final validation data set
FV_data = np.loadtxt('../data/test_data.txt',delimiter=' ',skiprows=1)
# Load input training data set
train_data = np.loadtxt('../data/training_data.txt',delimiter=' ',skiprows=1)
utrain_data = np.unique(train_data,axis=0)

# Get header words list
f = open('../data/test_data.txt','r')
words = np.array(f.readline().split())
f.close()

# Splot y_train and x_train from training set
x_tall = train_data[:,1:]
y_tall = train_data[:,0]

x_uall = utrain_data[:,1:]
y_uall = utrain_data[:,0]

# One hot encode categories
y_tall = keras.utils.np_utils.to_categorical(y_tall)

## Useful functions for Neural network debugging

In [ ]:
# Function to generate DNN of given depth and width
def getModel(layers,Pdrop):
    model = Sequential()
    model.add(Dense(layers[0],input_shape=(1000,)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(Pdrop))
    for i in layers[1:]:
        model.add(Dense(i))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    
    # predicting probabilities of each of the 2 classes
    model.add(Dense(2))
    model.add(Activation('softmax'))
    return model

# undo one hot encoding
def Unencode(out):
    ypred = out[:,0] < out[:,1]
    return ypred

# Function to get explicit model accuracy from softmax
def getAccuracy(model,xt,yt):
    out = model.predict(xt)
    ypred = Unencode(out)
    ytrue = Unencode(yt)
    acc = 1.0*np.sum(ypred == ytrue)/len(ytrue)
    return acc

# Function to get bag of words which were misclassified
def getBagOfWords(xtrain,ypred,ytrue,words):
    out = []
    ypredu = Unencode(ypred).astype(int)
    ytrueu = Unencode(ytrue).astype(int)
    # Get locations of bag of words which were misclassified
    idx = np.arange(0,len(ypredu))
    idxErr = idx[ypredu!=ytrueu]
    Xerr = xtrain[ypredu!=ytrueu]
    j = 0
    for i in Xerr:
        out.append([ytrue[idxErr[j]],ypred[idxErr[j]],words[i>0],i[i>0]])
        j=j+1
    return out

# Function to write final predictions
def writeResults(ypred):
    f = open('DNN_submission.txt','w')
    f.write('Id,Prediction\n')
    for i in range(0,len(ypred)):
        f.write(str(i+1)+','+str(int(ypred[i]))+'\n')
    f.close()

## Dense Neural network model for bag of words predictor

Initial testing of single DNN

In [72]:
# Split the training data k-fold number of ways for k-fold validation of the learning algorithm
k=5
kf = sklearn.model_selection.KFold(n_splits=k)
inds = [ind for ind in kf.split(x_tall, y_tall)]

i=0
train,val = inds[0]
# Training and validation data for k fold cross validation
Xtrain = x_tall[train]
Ytrain = y_tall[train]
Xval = x_tall[val]
Yval = y_tall[val]

# Define the DNN model
model = getModel([500,250,125],0.4)

# Compile it and fit
model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
model.fit(Xtrain, Ytrain, batch_size=2**8, epochs=4,verbose=1,validation_data=(Xval, Yval))
ypred = model.predict(Xval,batch_size=2**8,verbose=1)

Train on 16000 samples, validate on 4000 samples
Epoch 1/4
16000/16000 [==============================] - 9s 590us/step - loss: 0.4957 - acc: 0.7658 - val_loss: 0.3615 - val_acc: 0.8400
Epoch 2/4
16000/16000 [==============================] - 3s 206us/step - loss: 0.3180 - acc: 0.8627 - val_loss: 0.3440 - val_acc: 0.8553
Epoch 3/4
16000/16000 [==============================] - 3s 193us/step - loss: 0.2556 - acc: 0.8916 - val_loss: 0.3716 - val_acc: 0.8562
Epoch 4/4
4000/4000 [==============================] - 2s 526us/step


In [73]:
# Debugging bag of words
out = getBagOfWords(Xval,ypred,Yval,words)
print(out[0])

[array([0., 1.]), array([0.6994479 , 0.30055207], dtype=float32), array(['thi', 'veri', 'get', 'onli', 'look', 'want', 'see', 'mani', 'new',
       'peopl', 'still', 'need', 'two', 'feel', 'start', 'long', 'listen',
       'excel', 'enough', 'person', 'cover', 'almost', 'scene', 'instead',
       'famili', 'sever', 'hour', 'els', 'fine', 'talk', 'american',
       'entir', 'lack', 'impress', 'state', 'avail', 'certainli',
       'student', 'danc', 'parent', 'critic', 'centuri', 'train',
       'aspect'], dtype='<U12'), array([3., 2., 1., 1., 2., 1., 2., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 2., 2., 6., 1., 1., 1., 1., 1.])]


Method  validation for single DNN

In [137]:
# Compile the DNN model
def getDNNCross(k,x_tall,y_tall):
    # Storage for k fold cross validation    
    trainErr = []
    testErr = []
    acc = []
    
    # Split the training data k-fold number of ways for k-fold validation of the learning algorithm
    kf = sklearn.model_selection.KFold(n_splits=k)
    inds = [ind for ind in kf.split(x_tall, y_tall)]
    
    i=0
    for train,val in inds:
        # Training and validation data for k fold cross validation
        Xtrain = x_tall[train]
        Ytrain = y_tall[train]
        Xval = x_tall[val]
        Yval = y_tall[val]
    
        # Define the DNN model
        model = getModel([500,250,125],0.4)
        
        # Compile it and fit
        model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
        model.fit(Xtrain, Ytrain, batch_size=2**8, epochs=10,verbose=1,validation_data=(Xval, Yval))
        
        # store training and test error
        trainErr.append(model.evaluate(x=Xtrain, y=Ytrain))
        testErr.append(model.evaluate(x=Xval, y=Yval))
        acc.append(getAccuracy(model,Xval,Yval))

        # Status output
        print('k-iteration = ',i)
        i=i+1
        trainErr = np.array(trainErr)
        testErr = np.array(testErr)
        
    print('\n trainErr')
    print(trainErr)
    print('avg trainErr',np.mean(trainErr,axis=0))
    print('\n testErr')
    print(testErr)
    print('avg testErr',np.mean(testErr,axis=0))
    print('\n Accuracy')
    print(acc)
    print('avg acc',np.mean(acc))

In [138]:
getDNNCross(5,x_tall,y_tall)

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 14s 904us/step - loss: 0.4877 - acc: 0.7672 - val_loss: 0.3531 - val_acc: 0.8415
Epoch 2/10
16000/16000 [==============================] - 4s 232us/step - loss: 0.3224 - acc: 0.8587 - val_loss: 0.3569 - val_acc: 0.8460
Epoch 3/10
16000/16000 [==============================] - 3s 188us/step - loss: 0.2588 - acc: 0.8908 - val_loss: 0.3846 - val_acc: 0.8503
Epoch 4/10
16000/16000 [==============================] - 3s 183us/step - loss: 0.2115 - acc: 0.9119 - val_loss: 0.3895 - val_acc: 0.8470
Epoch 5/10
16000/16000 [==============================] - 3s 180us/step - loss: 0.1695 - acc: 0.9320 - val_loss: 0.4384 - val_acc: 0.8445
Epoch 6/10
16000/16000 [==============================] - 3s 202us/step - loss: 0.1379 - acc: 0.9449 - val_loss: 0.4329 - val_acc: 0.8460
Epoch 7/10
16000/16000 [==============================] - 3s 185us/step - loss: 0.1075 - acc: 0.9587 - val_loss: 0.5665 - 

AttributeError: 'numpy.ndarray' object has no attribute 'append'

Trying Random Forest of weakly trained Neural Networks

In [130]:
# Split the training data k-fold number of ways for k-fold validation of the learning algorithm
k=5
kf = sklearn.model_selection.KFold(n_splits=k)
inds = [ind for ind in kf.split(x_tall, y_tall)
train,val = inds[0]
# Training and validation data for k fold cross validation
Xtrain = x_uall[train]
Ytrain = y_uall[train]
Xval = x_uall[val]
Yval = y_uall[val]

# Specify number of Neural networks to train
N_models = 10
Predictions = []
TrainErr = []
TestErr = []

# Define the DNN model
for i in range(0,N_models):
    print('Training DNN ',i)
    model = getModel([500,250,125],0)
    # Compile it and fit
    model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
    model.fit(Xtrain, Ytrain, batch_size=2**8, epochs=2)
    # Store the models
    TrainErr.append(model.evaluate(x=Xtrain, y=Ytrain))
    TestErr.append(model.evaluate(x=Xval, y=Yval))
    # Use weakly trained model to predict and store predictions
    ypred = model.predict(Xval,batch_size=2**8)
    Predictions.append(ypred)

Predictions = np.array(Predictions)
TrainErr = np.array(TrainErr)
TestErr = np.array(TestErr)

Training DNN  0
Epoch 1/2
16000/16000 [==============================] - 9s 544us/step - loss: 0.4455 - acc: 0.7926
Epoch 2/2
4000/4000 [==============================] - 1s 196us/step
Training DNN  1
Epoch 1/2
16000/16000 [==============================] - 9s 565us/step - loss: 0.4401 - acc: 0.8002
Epoch 2/2
4000/4000 [==============================] - 1s 195us/step
Training DNN  2
Epoch 1/2
16000/16000 [==============================] - 10s 595us/step - loss: 0.4449 - acc: 0.7946
Epoch 2/2
4000/4000 [==============================] - 1s 247us/step
Training DNN  3
Epoch 1/2
16000/16000 [==============================] - 9s 575us/step - loss: 0.4316 - acc: 0.7963
Epoch 2/2
4000/4000 [==============================] - 1s 217us/step
Training DNN  4
Epoch 1/2
16000/16000 [==============================] - 10s 614us/step - loss: 0.4365 - acc: 0.7992
Epoch 2/2
4000/4000 [==============================] - 1s 232us/step
Training DNN  5
Epoch 1/2
16000/16000 [==============================] - 

16000/16000 [==============================] - 11s 668us/step - loss: 0.4343 - acc: 0.8021
Epoch 2/2
4000/4000 [==============================] - 1s 220us/step
Training DNN  25
Epoch 1/2
16000/16000 [==============================] - 10s 645us/step - loss: 0.4421 - acc: 0.7972
Epoch 2/2
4000/4000 [==============================] - 1s 222us/step
Training DNN  26
Epoch 1/2
16000/16000 [==============================] - 10s 649us/step - loss: 0.4364 - acc: 0.7961
Epoch 2/2
4000/4000 [==============================] - 1s 222us/step
Training DNN  27
Epoch 1/2
16000/16000 [==============================] - 11s 664us/step - loss: 0.4397 - acc: 0.7951
Epoch 2/2
4000/4000 [==============================] - 1s 235us/step
Training DNN  28
Epoch 1/2
16000/16000 [==============================] - 11s 663us/step - loss: 0.4388 - acc: 0.7949
Epoch 2/2
4000/4000 [==============================] - 1s 220us/step
Training DNN  29
Epoch 1/2
16000/16000 [==============================] - 11s 667us/step - l

16000/16000 [==============================] - 12s 749us/step - loss: 0.4262 - acc: 0.8036
Epoch 2/2
4000/4000 [==============================] - 1s 242us/step
Training DNN  49
Epoch 1/2
16000/16000 [==============================] - 12s 765us/step - loss: 0.4343 - acc: 0.7983
Epoch 2/2
4000/4000 [==============================] - 1s 247us/step


In [136]:
ypred = []
for i in Predictions:
    ypred.append(Unencode(i).astype(int))

# Get mean and standard deviation of samples
ypmean=np.mean(ypred,axis=0)
std=np.std(ypred,axis=0)

print('Number of samples where stdev > 0, ',np.sum(std>0))
ypred = (ypmean > 0.5).astype(int)
ytrue = Unencode(Yval).astype(int)

acc = 1.0*np.sum(ypred == ytrue)/len(ytrue)
print('Final accuracy of random forest = ',acc)

Number of samples where stdev > 0,  2119
Final accuracy of random forest =  0.86


In [142]:
# Random neuron forest
def getDNNForestCross(k,N_models,x_tall,y_tall):
    # Storage for k fold cross validation
    facc = []
    
    # Split the training data k-fold number of ways for k-fold validation of the learning algorithm
    k=5
    kf = sklearn.model_selection.KFold(n_splits=k)
    inds = [ind for ind in kf.split(x_tall, y_tall)]

    j=0
    for train,val in inds:
        print('Cross fold validation ',j,'/',k)
        # Training and validation data for k fold cross validation
        Xtrain = x_tall[train]
        Ytrain = y_tall[train]
        Xval = x_tall[val]
        Yval = y_tall[val]

        # Store predictions from each tree in DNN forest
        Predictions = []
        
        # Define the DNN model
        for i in range(0,N_models):
            print('Training DNN ',i)
            model = getModel([500,250,125],0)
            # Compile it and fit
            model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
            model.fit(Xtrain, Ytrain, batch_size=2**8, epochs=2,verbose=1)

            # Use weakly trained model to predict and store predictions
            ypred = model.predict(Xval,batch_size=2**8,verbose=1)
            Predictions.append(ypred)

        Predictions = np.array(Predictions)
        
        ypred = []
        for i in Predictions:
            ypred.append(Unencode(i).astype(int))

        # Get mean and standard deviation of samples
        ypmean=np.mean(ypred,axis=0)
        std=np.std(ypred,axis=0)
        print('Number of samples where stdev > 0, ',np.sum(std>0))
        
        # Compute accuracy of predictions
        ypred = (ypmean > 0.5).astype(int)
        ytrue = Unencode(Yval).astype(int)
        acc = 1.0*np.sum(ypred == ytrue)/len(ytrue)
        print('Final accuracy of random forest = ',acc)
        j=j+1
        
        facc.append(acc)
        
    print('avg Accuracy of all random forests',np.mean(facc))

In [ ]:
getDNNForestCross(5,10,x_tall,y_tall)

Cross fold validation  0 / 5
Training DNN  0
Epoch 1/2
16000/16000 [==============================] - 13s 797us/step - loss: 0.4342 - acc: 0.8000
Epoch 2/2
4000/4000 [==============================] - 5s 1ms/step
Training DNN  1
Epoch 1/2
16000/16000 [==============================] - 14s 844us/step - loss: 0.4393 - acc: 0.7969
Epoch 2/2
4000/4000 [==============================] - 5s 1ms/step
Training DNN  2
Epoch 1/2
16000/16000 [==============================] - 13s 813us/step - loss: 0.4315 - acc: 0.8037
Epoch 2/2
4000/4000 [==============================] - 5s 1ms/step
Training DNN  3
Epoch 1/2
16000/16000 [==============================] - 14s 849us/step - loss: 0.4482 - acc: 0.7908
Epoch 2/2
4000/4000 [==============================] - 5s 1ms/step
Training DNN  4
Epoch 1/2
16000/16000 [==============================] - 15s 914us/step - loss: 0.4264 - acc: 0.8039
Epoch 2/2
4000/4000 [==============================] - 6s 1ms/step
Training DNN  5
Epoch 1/2
16000/16000 [============

### Output final predictions from the model

In [ ]:
# Specify number of Neural networks to train
N_models = 20
Predictions = []

# Define the DNN model
for i in range(0,N_models):
    print('Training DNN ',i)
    model = getModel([500,250,125],0)
    # Compile it and fit
    model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
    model.fit(x_uall, y_uall, batch_size=2**8, epochs=2)
    # Use weakly trained model to predict and store predictions
    ypred = model.predict(FV_data,batch_size=2**8)
    Predictions.append(ypred)

In [ ]:
Predictions = np.array(Predictions)

ypred = []
for i in Predictions:
    ypred.append(Unencode(i).astype(int))

# Get mean and standard deviation of samples
ypmean=np.mean(ypred,axis=0)
std=np.std(ypred,axis=0)
print('Number of samples where stdev > 0, ',np.sum(std>0))

# Compute final predictions and output it
ypred = (ypmean > 0.5).astype(int)
writeResults(ypred)